In [37]:
# Imports and stuff
import os
import time
import csv

#Libraries
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction import stop_words
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold

#Powerpuff-stuff
import helpers
 


In [53]:
#TRYING TO FIND A BETTER STOP-WORD LIST

sklearn_stops = stop_words.ENGLISH_STOP_WORDS

custom_stop_words = set(sklearn_stops)

negation_words = set(["neither", "nor", "not", "either", "never", "no", "nobody", "nothing",
                      "nowhere", "none", "little", "few", "rarely", "hardly", "seldom", "only",
                      "can't", "cannot", "without", "noone"])

custom_stop_words = custom_stop_words - negation_words

len(custom_stop_words)

303

### Running for local testing


In [58]:
# RUNNING ON TRAINING SET WITH CROSS VALIDATION

training_set_pos = "train_pos.txt" 
training_set_neg = "train_neg.txt"
training_set_full = "train_full.txt"
test_set = "test_data.txt"

#Couning nr. of lines in pos and neg training_set
nr_pos_lines = 0
nr_neg_lines = 0

#Creating corpus ---------------------------------------
corpus = []

#Adding lines from positive training set to CORPUS
with open(training_set_pos,'rb') as infile:
    for line in infile:
        corpus.append(line)

nr_pos_lines = len(corpus)
        
#Adding lines from negative training set to CORPUS
with open(training_set_neg,'rb') as infile:
    for line in infile:
        corpus.append(line)
        
nr_neg_lines = len(corpus) - nr_pos_lines

#Finding tot nr. of lines
nr_lines_total = nr_pos_lines + nr_neg_lines

#Creating labels-array  ---------------------------------    

#Making labels, setting pos = 1 and neg = 0 
labels = np.zeros(nr_lines_total);
labels[0:nr_pos_lines]=1;
labels[nr_pos_lines:nr_lines_total]=0;

#Variables for measuring accuracy -----------------------

totalsvm = 0   # Accuracy measure 
totalMatSvm = np.zeros((2,2));  # Confusion matrix 

#Running algorithm with K-fold ---------------------------

#Creating K-fold = 10 for cross validation
kf = StratifiedKFold(n_splits=3)

for train_index, test_index in kf.split(corpus,labels):
    
    X_train = [corpus[i] for i in train_index]
    X_test = [corpus[i] for i in test_index]
    
    y_train, y_test = labels[train_index], labels[test_index]
    
    vectorizer = TfidfVectorizer(
        min_df=6, # removing word that occure less then 10 times 
        max_df = 1.3, # remove words that are too frequent ( more then 0.8 * number of tweets )
        sublinear_tf=True, # scale the term frequency in logarithmic scale
        use_idf =True
        #stop_words = custom_stop_words # Removing stop-words
    )
    
    train_corpus_tf_idf = vectorizer.fit_transform(X_train) 
    test_corpus_tf_idf = vectorizer.transform(X_test)

    model1 = LinearSVC()
    model1.fit(train_corpus_tf_idf,y_train)
    result1 = model1.predict(test_corpus_tf_idf)
    
    print(result1)
    
    totalMatSvm = totalMatSvm + confusion_matrix(y_test, result1)
    totalsvm = totalsvm+sum(y_test==result1)
    
print(totalMatSvm, "\n", "Total accuracy:", totalsvm/nr_lines_total)  

[ 0.  0.  1. ...,  0.  0.  0.]
[ 1.  1.  1. ...,  0.  0.  0.]
[ 0.  1.  0. ...,  0.  0.  0.]
[[ 76782.  23218.]
 [ 18263.  81737.]] 
 Total accuracy: 0.792595


### Running for keggle delivery

In [ ]:
start = time.time()

# DATA READING AND PREPROCESSING ------------------------------------------

training_set_pos = "train_pos_full.txt" 
training_set_neg = "train_neg_full.txt"

test_set = "test_data.txt"

#Couning nr. of lines in pos and neg training_set
nr_pos_lines = 0
nr_neg_lines = 0


#### CREATING CORPUS #### 
corpus = []

#Adding lines from positive training set to CORPUS
with open(training_set_pos,'rb') as infile:
    for line in infile:
        corpus.append(line)

nr_pos_lines = len(corpus)
print("pos_lines:", nr_pos_lines)
        
#Adding lines from negative training set to CORPUS
with open(training_set_neg,'rb') as infile:
    for line in infile:
        corpus.append(line)
        
nr_neg_lines = len(corpus) - nr_pos_lines
print("neg_lines:", nr_neg_lines)

nr_lines_total = nr_pos_lines + nr_neg_lines
print("total_lines:", nr_lines_total)

##########################


#### CREATING TEST CORPUS ####

test_corpus = []

with open(test_set,'rb') as infile:
    for line in infile:
        test_corpus.append(line)
        
        
##############################

#Making pos/neg labels, setting pos = 1 and neg = 0 
labels = np.zeros(nr_lines_total);
labels[0:nr_pos_lines]=1;
labels[nr_pos_lines:nr_lines_total]=-1;

vectorizer = TfidfVectorizer(
    min_df=6, # removing word that occure less then 10 times 
    max_df = 1.3, # remove words that are too frequent ( more then 0.8 * number of files )
    sublinear_tf=True, # scale the term frequency in logarithmic scale
    use_idf =True, 
    stop_words = 'english' # Removing stop-words
)

#CLASSIFYING WITH SVM
model = LinearSVC()

elapsed = time.time()

print ("Time in min:", (elapsed - start) / 60 )


In [ ]:
start = time.time()

#VECTORIZING ( Basically the preprocessing, see: TfidfVectorizer)
train_corpus_fittrans = vectorizer.fit_transform(corpus) 
test_corpus_trans = vectorizer.transform(test_corpus)

elapsed = time.time()

print ("Time in min:", (elapsed - start) / 60 )

In [ ]:
start = time.time()

# FITTING THE MODEL
model.fit(train_corpus_fittrans, labels)

elapsed = time.time()
print ("Time in min:", (elapsed - start) / 60 )

In [ ]:
start = time.time()

#PREDICTING
submission_prediction = model.predict(test_corpus_trans)

elapsed = time.time()
print ("Time in min:", (elapsed - start) / 60 )

In [ ]:
start = time.time()

#CREATING SUBMISSION
ids = list(range(1,nr_lines_total+1))
y_pred = submission_prediction
name = "keggle_submission_2.csv"

create_csv_submission(ids, y_pred, name)

elapsed = time.time()
print ("Time in min:", (elapsed - start) / 60 )

## Running on test-data with GRID-SEARCH on parameters for tokenizing 

In [3]:
# RUNNING ON TRAINING SET WITH CROSS VALIDATION

training_set_pos = "train_pos.txt" 
training_set_neg = "train_neg.txt"
test_set = "test_data.txt"

#Couning nr. of lines in pos and neg training_set
nr_pos_lines = 0
nr_neg_lines = 0

#Creating corpus ---------------------------------------
corpus = []

#Adding lines from positive training set to CORPUS
with open(training_set_pos,'rb') as infile:
    for line in infile:
        corpus.append(line)

nr_pos_lines = len(corpus)
        
#Adding lines from negative training set to CORPUS
with open(training_set_neg,'rb') as infile:
    for line in infile:
        corpus.append(line)
        
nr_neg_lines = len(corpus) - nr_pos_lines

#Finding tot nr. of lines
nr_lines_total = nr_pos_lines + nr_neg_lines

#Creating labels-array  ---------------------------------    

#Making labels, setting pos = 1 and neg = 0 
labels = np.zeros(nr_lines_total);
labels[0:nr_pos_lines]=1;
labels[nr_pos_lines:nr_lines_total]=0;

#Variables for measuring accuracy -----------------------

totalsvm = 0   # Accuracy measure 
totalMatSvm = np.zeros((2,2));  # Confusion matrix 

#Running algorithm with K-fold ---------------------------

 

In [11]:
min_df_list = list(range(10,13))
max_df_list = np.linspace(0.3, 2.0, num=15)

scores = []
best_score = 0
best_min_df = 0 
best_max_df = 0

for min_df in min_df_list:
    for max_df in max_df_list:
        
        #Creating K-fold for cross validation
        kf = StratifiedKFold(n_splits=2)
        
        totalsvm = 0

        for train_index, test_index in kf.split(corpus,labels):

            X_train = [corpus[i] for i in train_index]
            X_test = [corpus[i] for i in test_index]

            y_train, y_test = labels[train_index], labels[test_index]

            vectorizer = TfidfVectorizer(
                min_df = min_df, # removing word that occure less then X times 
                max_df = max_df, # remove words that are too frequent ( more then 0.8 * number of files )
                sublinear_tf=True, # scale the term frequency in logarithmic scale
                use_idf =True
                #stop_words = 'english', # Removing stop-words
            )

            train_corpus_tf_idf = vectorizer.fit_transform(X_train) 
            test_corpus_tf_idf = vectorizer.transform(X_test)

            model1 = LinearSVC()
            model1.fit(train_corpus_tf_idf,y_train)
            result1 = model1.predict(test_corpus_tf_idf)
            
                
            #totalMatSvm = totalMatSvm + confusion_matrix(y_test, result1)
            totalsvm = totalsvm+sum(y_test==result1)

        score = totalsvm / nr_lines_total
        scores.append(score)
        
        if(score > best_score):
            best_min_df = min_df
            best_max_df = max_df
            best_score = score
        
            print("Best score: ", best_score, "\n",
                  "With min_df:", min_df, " and ",
                  "Max_df:", max_df)
            

Best score:  0.789415 
 With min_df: 10  and  Max_df: 0.3


KeyboardInterrupt: 

## STUPID N-GRAM IMPLEMENTATION

In [39]:
start = time.time()
# RUNNING ON TRAINING SET WITH CROSS VALIDATION

training_set_pos = "train_pos_full.txt" 
training_set_neg = "train_neg_full.txt"
test_set = "test_data.txt"

#Couning nr. of lines in pos and neg training_set
nr_pos_lines = 0
nr_neg_lines = 0


def add_n_grams(tweet, n):

    n_grams = []
    words = tweet.split()
    for i in range(len(words)-1):        
        n_grams.append(b''.join([words[i], b'-', words[i+1]]))
    words.extend(n_grams)
    return b'-'.join(words)

#Creating corpus ---------------------------------------
corpus = []

n_gram = 2

#Adding lines from positive training set to CORPUS
with open(training_set_pos,'rb') as infile:
    for line in infile:
        line = add_n_grams(line, n_gram)
        corpus.append(line)

nr_pos_lines = len(corpus)
        
#Adding lines from negative training set to CORPUS
with open(training_set_neg,'rb') as infile:
    for line in infile:
        line = add_n_grams(line, n_gram)
        corpus.append(line)
        
nr_neg_lines = len(corpus) - nr_pos_lines

#Finding tot nr. of lines
nr_lines_total = nr_pos_lines + nr_neg_lines

#Creating labels-array  ---------------------------------    

#Making labels, setting pos = 1 and neg = 0 
labels = np.zeros(nr_lines_total);
labels[0:nr_pos_lines]=1;
labels[nr_pos_lines:nr_lines_total]=0;

#Variables for measuring accuracy -----------------------

totalsvm = 0   # Accuracy measure 
totalMatSvm = np.zeros((2,2));  # Confusion matrix 

#Running algorithm with K-fold ---------------------------
end = time.time()

print("time in min:", (end-start)/60)
 

time in min: 0.4770995577176412


In [40]:
start = time.time()

#### CREATING TEST CORPUS ####

test_corpus = []

with open(test_set,'rb') as infile:
    for line in infile:
        test_corpus.append(line)
        
        
##############################

#Making pos/neg labels, setting pos = 1 and neg = 0 
labels = np.zeros(nr_lines_total);
labels[0:nr_pos_lines]=1;
labels[nr_pos_lines:nr_lines_total]=-1;

vectorizer = TfidfVectorizer(
    min_df=10, # removing word that occure less then 10 times 
    max_df = 2.0, # remove words that are too frequent ( more then 0.8 * number of files )
    sublinear_tf=True, # scale the term frequency in logarithmic scale
    use_idf =True
    #stop_words = 'english' # Removing stop-words
)

end = time.time()

print("Time in min:", end-start)


Time in min: 0.06066179275512695


In [41]:
#CLASSIFYING WITH SVM
model = LinearSVC()

In [42]:
start = time.time()

#VECTORIZING ( Basically the preprocessing, see: TfidfVectorizer)
train_corpus_fittrans = vectorizer.fit_transform(corpus) 
test_corpus_trans = vectorizer.transform(test_corpus)

elapsed = time.time()

print ("Time in min:", (elapsed - start) / 60 )

Time in min: 1.4847740610440572


In [43]:
start = time.time()

# FITTING THE MODEL
model.fit(train_corpus_fittrans, labels)

elapsed = time.time()
print ("Time in min:", (elapsed - start) / 60 )

Time in min: 14.095175957679748


In [44]:
start = time.time()

#PREDICTING
submission_prediction = model.predict(test_corpus_trans)

elapsed = time.time()
print ("Time in min:", (elapsed - start) / 60 )

Time in min: 4.179080327351888e-05


In [47]:
start = time.time()

#CREATING SUBMISSION
ids = list(range(1,nr_lines_total+1))
y_pred = submission_prediction
name = "keggle_submission_3.csv"

create_csv_submission(ids, y_pred, name)

elapsed = time.time()
print ("Time in min:", (elapsed - start) / 60 )

Time in min: 0.004236304759979248


In [46]:
def create_csv_submission(ids, y_pred, name):
    """
    Creates an output file in csv format for submission to kaggle
    Arguments: ids (event ids associated with each prediction)
               y_pred (predicted class labels)
               name (string name of .csv output file to be created)
    """
    with open(name, 'w') as csvfile:
        fieldnames = ['Id', 'Prediction']
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
        writer.writeheader()
        for r1, r2 in zip(ids, y_pred):
            writer.writerow({'Id':int(r1),'Prediction':int(r2)})